In [1]:
import pandas as pd
import numpy as np 
data = pd.read_feather('database/base.feather')

In [2]:
data

,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,City,State,Region,...,OMonth,OPeriod,Order Date Month,SYear,SMonth,SPeriod,Ship Date Month,Delivery,Price,Benefit
0,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,Henderson,Kentucky,South,...,11,35,2016-11-01,2016,11,35,2016-11-01,3,130.98,1
1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,Henderson,Kentucky,South,...,11,35,2016-11-01,2016,11,35,2016-11-01,3,243.98,1
2,CA-2016-138688,2016-06-12,2016-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,Los Angeles,California,West,...,6,30,2016-06-01,2016,6,30,2016-06-01,4,7.31,1
3,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,Fort Lauderdale,Florida,South,...,10,22,2015-10-01,2015,10,22,2015-10-01,7,191.52,0
4,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,Fort Lauderdale,Florida,South,...,10,22,2015-10-01,2015,10,22,2015-10-01,7,11.18,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9989,CA-2014-110422,2014-01-21,2014-01-23,Second Class,TB-21400,Tom Boeckenhauer,Consumer,Miami,Florida,South,...,1,1,2014-01-01,2014,1,1,2014-01-01,2,8.42,1
9990,CA-2017-121258,2017-02-26,2017-03-03,Standard Class,DB-13060,Dave Brooks,Consumer,Costa Mesa,California,West,...,2,38,2017-02-01,2017,3,39,2017-03-01,5,45.98,1
9991,CA-2017-121258,2017-02-26,2017-03-03,Standard Class,DB-13060,Dave Brooks,Consumer,Costa Mesa,California,West,...,2,38,2017-02-01,2017,3,39,2017-03-01,5,129.29,1
9992,CA-2017-121258,2017-02-26,2017-03-03,Standard Class,DB-13060,Dave Brooks,Consumer,Costa Mesa,California,West,...,2,38,2017-02-01,2017,3,39,2017-03-01,5,7.40,1


In [3]:
def zscore(df, grupo, var, score):
    media = df[var].mean()
    dpadr = df[var].std()
    gr = df.groupby(grupo)[var]
    dc = gr.describe()
    dc = dc.reset_index()
    dc[score] = dc['mean'].apply(lambda x : (x - media) / dpadr)
    dcz = dc[[grupo, score]]
    df = df.merge(dcz, on=grupo, how='left')
    return df

In [7]:
## Classificacao zcore
variaveis_dimensoes = ['Segment', 'Region', 'Category', 'Sub-Category', 'OYear', 'OMonth']

for dimensao in variaveis_dimensoes:
    print('Dimensão ==> {}'.format(dimensao))
    medidas = ['Sales', 'Profit', 'Quantity', 'Discount', 'Delivery', 'Price']
    grupo = data.groupby(dimensao)[medidas].mean().reset_index()
    for medida in medidas: 
        grupo = zscore(grupo, dimensao, medida, 'z'+medida)
    zmedidas = ['z'+med for med in medidas]
    zgrupo = grupo[zmedidas]
    grupo['score'] = zgrupo.sum(axis=1)
    media_score = grupo['score'].mean()
    dpadr_score = grupo['score'].std()
    grupo['classe'] = grupo['score'].apply(lambda x : int((x - media_score) / dpadr_score) + 3)
    grupo['rank'] = grupo['score'].rank(ascending=False)
    grupo['lucro'] = grupo['Profit'].apply(lambda x : 0 if x < 0 else 1)
    grupo.to_feather('database/classificacaoz_{}.feather'.format(dimensao))

Dimensão ==> Segment
Dimensão ==> Region
Dimensão ==> Category
Dimensão ==> Sub-Category
Dimensão ==> OYear
Dimensão ==> OMonth
